**Intro**

Often, we can or need to scrape information from the Internet. Let's learn how to clean this data up (if need be) and pass it to Pandas.

This notebook should be treated more as lecture notes than an exercise set.

##### How else can we get information from the web?
- Scraping the web
- APIs (Mostly JSON)

### How talking over the internet works

1. Client - (request) ---> Server
1. Server - (response) ---> Client

What does a request look like? 

```
GET /index.html HTTP/1.1
User-Agent: Mozilla/4.0 (compatible; MSIE5.01; Windows NT)
Host: www.google.com
Accept-Language: en-us
Accept-Encoding: gzip, deflate
Connection: Keep-Alive
```

What does a response look like?

```html
<!doctype html>
<html lang="en">
    <head>
        <meta charset=utf-8>
        <title>My Title</title>
    </head>
    <body>
        <p>I'm some content.</p>
        <p>
            <ol>
                <li>I'm a <a href="https://www.google.com">link</a>!</li>
                <li>I'm a <a href="https://www.google.com">link</a>!</li>
                <li>I'm a <a href="https://www.google.com">link</a>!</li>
            </ol>
        </p>
    </body>
</html>
```

#### Requests in Python

In [1]:
import requests  # Defacto web request library in Python ecosystem

# The requests library will make send a GET request to wikipedia with a GET line of:
#    GET /wiki/Columbia_University HTTP/1.1
#    Host: en.wikipedia.org
response = requests.get('https://en.wikipedia.org/wiki/Columbia_University')  
response

<Response [200]>

In [2]:
response.content

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Columbia University - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";var cookie=document.cookie.match(/(?:^|; )enwikimwclientpre

In [3]:
import bs4  # Beautiful Soup library helps us parse the content into objects. 

soup = bs4.BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Columbia University - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";var cookie=document.cookie.match(/(?:^|; )enwikimwclientpreferen

In [4]:
soup.find_all('p')  # Give us all the link elements as link objects.

[<p class="mw-empty-elt">
 </p>,
 <p><b>Columbia University</b>, officially titled as <b>Columbia University in the City of New York</b>, is a <a href="/wiki/Private_university" title="Private university">private</a> <a href="/wiki/Ivy_League" title="Ivy League">Ivy League</a> <a href="/wiki/Research_university" title="Research university">research university</a> in <a href="/wiki/New_York_City" title="New York City">New York City</a>. Established in 1754 as <b>King's College</b> on the grounds of <a href="/wiki/Trinity_Church_(Manhattan)" title="Trinity Church (Manhattan)">Trinity Church</a> in <a href="/wiki/Manhattan" title="Manhattan">Manhattan</a>, it is the oldest institution of higher education in <a href="/wiki/New_York_(state)" title="New York (state)">New York</a> and <a href="/wiki/First_university_in_the_United_States" title="First university in the United States">the fifth-oldest in the United States</a>.
 </p>,
 <p>Columbia was established as a <a href="/wiki/Colonial_col

#### Getting and cleaning HTML data text from paragraphs

In [5]:
soup.find_all('p')[0].text

'\n'

In [6]:
# Get all elements of type "p"
paragraph_elements = soup.find_all('p')
paragraph_elements

[<p class="mw-empty-elt">
 </p>,
 <p><b>Columbia University</b>, officially titled as <b>Columbia University in the City of New York</b>, is a <a href="/wiki/Private_university" title="Private university">private</a> <a href="/wiki/Ivy_League" title="Ivy League">Ivy League</a> <a href="/wiki/Research_university" title="Research university">research university</a> in <a href="/wiki/New_York_City" title="New York City">New York City</a>. Established in 1754 as <b>King's College</b> on the grounds of <a href="/wiki/Trinity_Church_(Manhattan)" title="Trinity Church (Manhattan)">Trinity Church</a> in <a href="/wiki/Manhattan" title="Manhattan">Manhattan</a>, it is the oldest institution of higher education in <a href="/wiki/New_York_(state)" title="New York (state)">New York</a> and <a href="/wiki/First_university_in_the_United_States" title="First university in the United States">the fifth-oldest in the United States</a>.
 </p>,
 <p>Columbia was established as a <a href="/wiki/Colonial_col

In [7]:
# Normalize text from each paragraph.
texts = [p.text.strip().lower() for p in paragraph_elements]
texts

['',
 "columbia university, officially titled as columbia university in the city of new york, is a private ivy league research university in new york city. established in 1754 as king's college on the grounds of trinity church in manhattan, it is the oldest institution of higher education in new york and the fifth-oldest in the united states.",
 'columbia was established as a colonial college by royal charter under george ii of great britain. it was renamed columbia college in 1784 following the american revolution, and in 1787 was placed under a private board of trustees headed by former students alexander hamilton and john jay. in 1896, the campus was moved to its current location in morningside heights and renamed columbia university.',
 "columbia is organized into twenty schools, including four undergraduate schools and 16 graduate schools. the university's research efforts include the lamont–doherty earth observatory, the goddard institute for space studies, and accelerator labora

In [8]:
# Replace all whitespace with a sinlge space character
text = ' '.join(texts)
text

' columbia university, officially titled as columbia university in the city of new york, is a private ivy league research university in new york city. established in 1754 as king\'s college on the grounds of trinity church in manhattan, it is the oldest institution of higher education in new york and the fifth-oldest in the united states. columbia was established as a colonial college by royal charter under george ii of great britain. it was renamed columbia college in 1784 following the american revolution, and in 1787 was placed under a private board of trustees headed by former students alexander hamilton and john jay. in 1896, the campus was moved to its current location in morningside heights and renamed columbia university. columbia is organized into twenty schools, including four undergraduate schools and 16 graduate schools. the university\'s research efforts include the lamont–doherty earth observatory, the goddard institute for space studies, and accelerator laboratories with

In [9]:
import re

# Let's remove some of the "[number]" noise...
text = re.sub(r'\[\d+\]', '', text)
text

' columbia university, officially titled as columbia university in the city of new york, is a private ivy league research university in new york city. established in 1754 as king\'s college on the grounds of trinity church in manhattan, it is the oldest institution of higher education in new york and the fifth-oldest in the united states. columbia was established as a colonial college by royal charter under george ii of great britain. it was renamed columbia college in 1784 following the american revolution, and in 1787 was placed under a private board of trustees headed by former students alexander hamilton and john jay. in 1896, the campus was moved to its current location in morningside heights and renamed columbia university. columbia is organized into twenty schools, including four undergraduate schools and 16 graduate schools. the university\'s research efforts include the lamont–doherty earth observatory, the goddard institute for space studies, and accelerator laboratories with

In [10]:
# Could use "replace" that would require us to use a bit of code for each odd pattern.
text = text.replace('\xa0', ' ')
text

' columbia university, officially titled as columbia university in the city of new york, is a private ivy league research university in new york city. established in 1754 as king\'s college on the grounds of trinity church in manhattan, it is the oldest institution of higher education in new york and the fifth-oldest in the united states. columbia was established as a colonial college by royal charter under george ii of great britain. it was renamed columbia college in 1784 following the american revolution, and in 1787 was placed under a private board of trustees headed by former students alexander hamilton and john jay. in 1896, the campus was moved to its current location in morningside heights and renamed columbia university. columbia is organized into twenty schools, including four undergraduate schools and 16 graduate schools. the university\'s research efforts include the lamont–doherty earth observatory, the goddard institute for space studies, and accelerator laboratories with

In [11]:
text.split()

['columbia',
 'university,',
 'officially',
 'titled',
 'as',
 'columbia',
 'university',
 'in',
 'the',
 'city',
 'of',
 'new',
 'york,',
 'is',
 'a',
 'private',
 'ivy',
 'league',
 'research',
 'university',
 'in',
 'new',
 'york',
 'city.',
 'established',
 'in',
 '1754',
 'as',
 "king's",
 'college',
 'on',
 'the',
 'grounds',
 'of',
 'trinity',
 'church',
 'in',
 'manhattan,',
 'it',
 'is',
 'the',
 'oldest',
 'institution',
 'of',
 'higher',
 'education',
 'in',
 'new',
 'york',
 'and',
 'the',
 'fifth-oldest',
 'in',
 'the',
 'united',
 'states.',
 'columbia',
 'was',
 'established',
 'as',
 'a',
 'colonial',
 'college',
 'by',
 'royal',
 'charter',
 'under',
 'george',
 'ii',
 'of',
 'great',
 'britain.',
 'it',
 'was',
 'renamed',
 'columbia',
 'college',
 'in',
 '1784',
 'following',
 'the',
 'american',
 'revolution,',
 'and',
 'in',
 '1787',
 'was',
 'placed',
 'under',
 'a',
 'private',
 'board',
 'of',
 'trustees',
 'headed',
 'by',
 'former',
 'students',
 'alexander',


#### Descriptive Statistics

In [12]:
import collections

counter = collections.Counter(text.split())

In [13]:
counter.most_common(20)

[('the', 803),
 ('of', 526),
 ('and', 364),
 ('in', 258),
 ('columbia', 196),
 ('a', 166),
 ('to', 140),
 ('for', 102),
 ('university', 94),
 ('is', 93),
 ('by', 86),
 ('as', 84),
 ('was', 79),
 ('college', 75),
 ('school', 75),
 ('new', 73),
 ('on', 56),
 ('with', 56),
 ('united', 52),
 ('at', 50)]

In [14]:
!pip3 install nltk  # Install natrual language toolkit

In [15]:
import nltk; nltk.download()
# If GUI does not show up: d, Enter, stopwords, Enter, q
# If GUI does show up: Corpa > double click Stopwords

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [16]:
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [17]:
counts = [(word, count) for word, count in counter.most_common() 
          if word not in stops]
counts

[('columbia', 196),
 ('university', 94),
 ('college', 75),
 ('school', 75),
 ('new', 73),
 ('united', 52),
 ('president', 48),
 ('first', 41),
 ('york', 40),
 ('students', 38),
 ('—', 33),
 ('undergraduate', 32),
 ('arts', 32),
 ('graduate', 31),
 ('include', 28),
 ('states', 26),
 ('campus', 24),
 ('world', 24),
 ('u.s.', 22),
 ('student', 22),
 ('including', 21),
 ("columbia's", 20),
 ('barnard', 19),
 ('university.', 18),
 ("university's", 18),
 ('also', 18),
 ('sciences', 18),
 ('general', 18),
 ('science', 18),
 ('university,', 17),
 ('city', 17),
 ('research', 17),
 ('john', 15),
 ('nobel', 15),
 ('founded', 15),
 ('ranked', 15),
 ("king's", 14),
 ('oldest', 14),
 ('academic', 14),
 ('faculty', 14),
 ('former', 13),
 ('prize', 13),
 ('college.', 13),
 ('state', 13),
 ('established', 12),
 ('american', 12),
 ('founding', 12),
 ('national', 12),
 ('college,', 12),
 ('columbia,', 12),
 ('many', 12),
 ('one', 12),
 ('journal', 12),
 ('morningside', 11),
 ('schools', 11),
 ('library',

In [ ]:
[(word, count) for word, count in reversed(counts) if count > 1]  # 'protests' only popped up 3 times...

### Exercise

Find the most common words and least common words on a few of your favorite websites. 

#### Getting JSON Data
So much less cleaning!

In [18]:
# https://data.bts.gov/Aviation/Commercial-Aviation-Departures/bpqk-hyst
# Number of commercial aviation departures, domestic and international, by day. 
# Also the number of people screened by TSA in total and at large hub airports by day.
import requests

response = requests.get('https://data.bts.gov/resource/bpqk-hyst.json')  
response

<Response [200]>

In [19]:
response.json()[:5]

[{'mode': 'Aviation',
  'indicator': 'U.S. Domestic Commercial Flights (Departures)',
  'date': '2021-08-01T00:00:00.000',
  'week_num': '32.21',
  'lowest': '6679',
  'current': '23553',
  'baseline': '26791'},
 {'mode': 'Aviation',
  'indicator': 'U.S. Domestic Commercial Flights (Departures)',
  'date': '2021-08-02T00:00:00.000',
  'week_num': '32.21',
  'lowest': '6679',
  'current': '23840',
  'baseline': '28337'},
 {'mode': 'Aviation',
  'indicator': 'U.S. Domestic Commercial Flights (Departures)',
  'date': '2021-08-03T00:00:00.000',
  'week_num': '32.21',
  'lowest': '6679',
  'current': '23705',
  'baseline': '28660'},
 {'mode': 'Aviation',
  'indicator': 'U.S. Domestic Commercial Flights (Departures)',
  'date': '2021-08-04T00:00:00.000',
  'week_num': '32.21',
  'lowest': '6679',
  'current': '24411',
  'baseline': '27821'},
 {'mode': 'Aviation',
  'indicator': 'U.S. Domestic Commercial Flights (Departures)',
  'date': '2021-08-05T00:00:00.000',
  'week_num': '32.21',
  'low

In [20]:
set(i['mode'] for i in response.json())

{'Aviation'}

In [21]:
set(i['indicator'] for i in response.json())

{'People Screened at All Airports',
 'U.S. Domestic Commercial Flights (Departures)',
 'U.S. International Commercial Flights (Departures)'}

In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame(response.json())
df

,mode,indicator,date,week_num,lowest,current,baseline
0,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-01T00:00:00.000,32.21,6679,23553,26791
1,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-02T00:00:00.000,32.21,6679,23840,28337
2,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-03T00:00:00.000,32.21,6679,23705,28660
3,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-04T00:00:00.000,32.21,6679,24411,27821
4,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-05T00:00:00.000,32.21,6679,25241,29244
...,...,...,...,...,...,...,...
995,Aviation,U.S. International Commercial Flights (Departu...,2021-04-10T00:00:00.000,15.21,326,2220,4767
996,Aviation,U.S. International Commercial Flights (Departu...,2021-04-11T00:00:00.000,16.21,326,2046,4434
997,Aviation,U.S. International Commercial Flights (Departu...,2021-04-12T00:00:00.000,16.21,326,1889,4311
998,Aviation,U.S. International Commercial Flights (Departu...,2021-04-13T00:00:00.000,16.21,326,1759,4158


In [24]:
# But wait? Why only 1000 rows? https://dev.socrata.com/docs/paging.html
response = requests.get(
    'https://data.bts.gov/resource/bpqk-hyst.json',
    params={
        '$limit': '3000',
    }
)
df = pd.DataFrame(response.json())
df

,mode,indicator,date,week_num,lowest,current,baseline
0,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-01T00:00:00.000,32.21,6679,23553,26791
1,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-02T00:00:00.000,32.21,6679,23840,28337
2,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-03T00:00:00.000,32.21,6679,23705,28660
3,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-04T00:00:00.000,32.21,6679,24411,27821
4,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-05T00:00:00.000,32.21,6679,25241,29244
...,...,...,...,...,...,...,...
2436,Aviation,U.S. International Commercial Flights (Departu...,2021-03-23T00:00:00.000,13.21,326,1679,4150
2437,Aviation,U.S. International Commercial Flights (Departu...,2021-03-24T00:00:00.000,13.21,326,1824,4347
2438,Aviation,U.S. International Commercial Flights (Departu...,2021-03-25T00:00:00.000,13.21,326,1895,4448
2439,Aviation,U.S. International Commercial Flights (Departu...,2021-03-26T00:00:00.000,13.21,326,1970,4517


In [25]:
df['date']  # Hmmm, type is "object". We want that to be something like "datetime".

0       2021-08-01T00:00:00.000
1       2021-08-02T00:00:00.000
2       2021-08-03T00:00:00.000
3       2021-08-04T00:00:00.000
4       2021-08-05T00:00:00.000
                 ...           
2436    2021-03-23T00:00:00.000
2437    2021-03-24T00:00:00.000
2438    2021-03-25T00:00:00.000
2439    2021-03-26T00:00:00.000
2440    2021-03-27T00:00:00.000
Name: date, Length: 2441, dtype: object

In [26]:
df['date'] = pd.to_datetime(df['date'])  # Another way to cast all date times in a series.
df  # FYI, we've done this before!

,mode,indicator,date,week_num,lowest,current,baseline
0,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-01,32.21,6679,23553,26791
1,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-02,32.21,6679,23840,28337
2,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-03,32.21,6679,23705,28660
3,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-04,32.21,6679,24411,27821
4,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-05,32.21,6679,25241,29244
...,...,...,...,...,...,...,...
2436,Aviation,U.S. International Commercial Flights (Departu...,2021-03-23,13.21,326,1679,4150
2437,Aviation,U.S. International Commercial Flights (Departu...,2021-03-24,13.21,326,1824,4347
2438,Aviation,U.S. International Commercial Flights (Departu...,2021-03-25,13.21,326,1895,4448
2439,Aviation,U.S. International Commercial Flights (Departu...,2021-03-26,13.21,326,1970,4517


In [27]:
df['current']

0       23553
1       23840
2       23705
3       24411
4       25241
        ...  
2436     1679
2437     1824
2438     1895
2439     1970
2440     2250
Name: current, Length: 2441, dtype: object

In [28]:
df['current'] = df['current'].astype(int)
df

,mode,indicator,date,week_num,lowest,current,baseline
0,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-01,32.21,6679,23553,26791
1,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-02,32.21,6679,23840,28337
2,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-03,32.21,6679,23705,28660
3,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-04,32.21,6679,24411,27821
4,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-05,32.21,6679,25241,29244
...,...,...,...,...,...,...,...
2436,Aviation,U.S. International Commercial Flights (Departu...,2021-03-23,13.21,326,1679,4150
2437,Aviation,U.S. International Commercial Flights (Departu...,2021-03-24,13.21,326,1824,4347
2438,Aviation,U.S. International Commercial Flights (Departu...,2021-03-25,13.21,326,1895,4448
2439,Aviation,U.S. International Commercial Flights (Departu...,2021-03-26,13.21,326,1970,4517


In [29]:
df_domestic = df[df['indicator'].str.contains('Domestic')]
df_domestic

,mode,indicator,date,week_num,lowest,current,baseline
0,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-01,32.21,6679,23553,26791
1,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-02,32.21,6679,23840,28337
2,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-03,32.21,6679,23705,28660
3,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-04,32.21,6679,24411,27821
4,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-05,32.21,6679,25241,29244
...,...,...,...,...,...,...,...
1984,Aviation,U.S. Domestic Commercial Flights (Departures),2021-03-23,13.21,6679,19289,27219
1985,Aviation,U.S. Domestic Commercial Flights (Departures),2021-03-24,13.21,6679,19856,27560
1986,Aviation,U.S. Domestic Commercial Flights (Departures),2021-03-25,13.21,6679,21662,28209
1987,Aviation,U.S. Domestic Commercial Flights (Departures),2021-03-26,13.21,6679,21217,27648


In [30]:
from bokeh.plotting import figure, show, output_notebook

output_notebook()

Loading BokehJS ...

In [31]:
p = figure(
    title='Airport Traffic', 
    x_axis_label='Date',
    y_axis_label='Departures',
    x_axis_type='datetime',
    width=900,
)

p.line(
    df_domestic['date'], 
    df_domestic['current'], 
    line_width=2,
    color='blue',
)

show(p)

In [32]:
df_domestic.head(25)

,mode,indicator,date,week_num,lowest,current,baseline
0,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-01,32.21,6679,23553,26791
1,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-02,32.21,6679,23840,28337
2,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-03,32.21,6679,23705,28660
3,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-04,32.21,6679,24411,27821
4,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-05,32.21,6679,25241,29244
5,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-06,32.21,6679,24759,29076
6,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-07,32.21,6679,21795,23453
14,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-08,33.21,6679,24034,26717
15,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-09,33.21,6679,24059,27871
16,Aviation,U.S. Domestic Commercial Flights (Departures),2021-08-10,33.21,6679,24062,28491


In [33]:
df = df.sort_values(by=['date'])
df

,mode,indicator,date,week_num,lowest,current,baseline
1310,Aviation,U.S. Domestic Commercial Flights (Departures),2020-01-01,1,NaN,21801,27113
1330,Aviation,People Screened at All Airports,2020-01-01,1,NaN,2155574,2354510
1989,Aviation,U.S. International Commercial Flights (Departu...,2020-01-01,1,NaN,4241,4709
1311,Aviation,U.S. Domestic Commercial Flights (Departures),2020-01-02,1,NaN,27239,26388
1331,Aviation,People Screened at All Airports,2020-01-02,1,NaN,2415487,2211239
...,...,...,...,...,...,...,...
1570,Aviation,People Screened at All Airports,2022-03-25,13.22,88341,2201949,2464077
792,Aviation,U.S. International Commercial Flights (Departu...,2022-03-25,13.22,NaN,3249,4517
751,Aviation,U.S. Domestic Commercial Flights (Departures),2022-03-26,13.22,NaN,21106,22324
1571,Aviation,People Screened at All Airports,2022-03-26,13.22,88341,1919916,2106402


In [34]:
df_domestic = df[df['indicator'].str.contains('Domestic')]
df_international = df[df['indicator'].str.contains('International')]

In [35]:
from bokeh.models import HoverTool

p = figure(
    title='Airport Traffic', 
    x_axis_label='Date',
    y_axis_label='Departures',
    x_axis_type='datetime',
    width=900,
)

p.line(
    df_domestic['date'], 
    df_domestic['current'], 
    line_width=2,
    color='blue',
    legend_label='domestic',
)

p.line(
    df_international['date'], 
    df_international['current'], 
    line_width=2,
    color='red',
    legend_label='international',
)

p.add_tools(HoverTool(
    tooltips=[
        ('Date',   '$x{%F}'),
        ('Count',  '$y{i}'),
    ],

    formatters={
        '$x': 'datetime',
    },

    mode='vline',
))

p.legend.location = "top_right"

show(p)

In [36]:
# Fridays are high traffic. Weekends are very low. 
# 2020-03-13, sharp drop
# 2021-06-25

#### Domestic Dip

In [37]:
import datetime

pre_pandemic = df_domestic[df_domestic['date'] < datetime.datetime(2020, 3, 13)]
post_vaccine = df_domestic[df_domestic['date'] > datetime.datetime(2021, 6, 25)]

In [38]:
100 * (post_vaccine['current'].mean() - pre_pandemic['current'].mean()) / pre_pandemic['current'].mean()

-9.364827892565158

#### International Dip

In [39]:
pre_pandemic = df_international[df_international['date'] < datetime.datetime(2020, 3, 13)]
post_vaccine = df_international[df_international['date'] > datetime.datetime(2021, 6, 25)]

In [40]:
100 * (post_vaccine['current'].mean() - pre_pandemic['current'].mean()) / pre_pandemic['current'].mean()

-30.49832033294375